Introduction to Tensorflow

In [6]:
import tensorflow as tf

# constant nodes -> they don't have an input - example : a bias
node1 = tf.constant(3.0, dtype=tf.float32, name="node1")
node2 = tf.constant(4.0, name="blablahhh") # also tf.float32 implicitly
node3 = tf.constant(5.0)

# this will print the nodes' characteristics not the constants
print(node1, node2, node3)

Tensor("node1_3:0", shape=(), dtype=float32) Tensor("blablahhh_3:0", shape=(), dtype=float32) Tensor("Const_3:0", shape=(), dtype=float32)


In [7]:
# let's start a session and evaluate the computational graph :
sess = tf.Session()
print(sess.run([node1, node2, node3])) # will print the node's outputs

[3.0, 4.0, 5.0]


We can build more complicated computations by combining Tensor nodes with operations (Operations are also nodes). 
For example, we can add our two constant nodes and produce a new graph as follows:

In [8]:
from __future__ import print_function
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))

node3: Tensor("Add_1:0", shape=(), dtype=float32)
sess.run(node3): 7.0


As it stands, this graph is not especially interesting because it always produces a constant result. A graph can be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a value later.

In [9]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

The preceding three lines are a bit like a function or a lambda in which we define two input parameters (a and b) and then an operation on them. We can evaluate this graph with multiple inputs by using the feed_dict argument to the run method to feed concrete values to the placeholders:

In [10]:
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

7.5
[ 3.  7.]
